# Artificial Intelligence Spring 99 <img src = 'https://ece.ut.ac.ir/cict-theme/images/footer-logo.png' alt="Tehran-University-Logo" width="150" height="150" align="right">
## Project 2: Genetic Algorithms
### Dr. Hakimeh Fadaei
### By Omid Vaheb

### Introduction:
In this project I implemented Generic Algorithm using a simple fitness function on decrypting substitution cypher problem. Like any other genetic algorithm methods chosen for building population, crossover, mutation and chromosomes were very important and vital.

### Question:
The question in this project is to decrypt a text given to us that we know is a result of cypher substitution ran on an unknown text. This kind of encryption replaces every letter in text with another letter using a key which relate each letter of alphabet to another one.
### Solution:
#### Chromosome:
I used Genetic Algorithm to solve this problem and in the process at first, I had to define chromosome. I defined a class that had 2 features named data and fitness. Data is a string including every letter of alphabet in 26 places and it encodes that the first letter in this string is going to replace letter in text and second one will replace b and so on. The other feature is fitness that will be calculated using decodeText function. 
#### Fitness Function:
It decodes given text using chromosome's data string and then calculates how many words of it is known for us using a defined dictionary. Fitness in this solution is number of known words for us in the decrypted text.
#### Dictionary:
This dictionary is the outcome of word extraction from global_text.txt file, which is a text promised to include every word of original text so if we find the correct encryption key, every word of decrypted text would be in this dictionary. I also implemented this dictionary using set and hash for a good time complexity. An important part of word extraction is data cleaning. In this project we got the dirty data and seperated it's words using any non-alphabetic character and making them all lowercase. The other problem is that some of words are misspelled or sticked together but there isn't any way for us to correct them without sing another valid dictionary.
#### First Population:
makePopulation function makes the first population using sample function of random library. Size of population in this question should be static and not dynamic although some experiments have showed that dynamic size will result in better performance but in this problem it not only doesn't help us but also increases the time of each generation at the end. In this problem every step we get closer to the answer fewer changes are needed to reach answer because only few characters need to change and increasing population size would be wasteless. In this case the optimal population size is between 400 and 600.

Now we import libraries needed in the text and write the fitness function:

In [1]:
import re
import random
import time
import string
import operator
def decodeText(letters, text, dictionary):
    decodedText = ""
    for character in text:
        if character.isalpha():
            decodedText += letters[ord(character.lower()) - ord('a')]
        else:
            decodedText += character
    words = re.split('\W', decodedText)
    score = 0
    for word in words:
        if word:
            if hash(word) in dictionary:
                score += 1
    return score

#### Decoder:
The next step is to complete Decoder class by writing it's main function, decode. This function will generate the next generation from current population using mutation, crossover and elite selection. It will generate new populations until one of the chromosomes reach the maximum fitness which is calculate by wordsCounter function and is equal to number of words available in the text. now let's get into details:
#### Mutation:
The act of mutation used in this problem is simple. It swaps two charaters in the chromosomes data ba a probabilty of mutationP. By choosing this probabilty between 0.15 and 0.4, algorithm will result in correct answer.
#### Crossover:
For crossover I used many methods such as coin flip for each gene or producing each child using half of chromosome of each parent but the one that actually worked was too choose a random part of each parent with a random size and fill the other genes using the other parent's chromosome. The probability of happening of this act equals to crossoverP and will result in answer in range of 0.6 to 0.8. The other important point about crossover is to build the mating pool regarding to giving more chance to chromosomes that have higher fitnesses. For this purpose I sorted population fitness-wise and builded a probability pool by adding index times the value of index of each chromosome in population to a list and choose random-wise from it.

We cn't say which one of crossover and mutation is better because each one is better in specific situations and the both help us to reach answer. Crossover could chand all 26 genes of a chromosome and is perfet for the first stages that we are far away from answer and mutation helps us to pass local maximums by changing only 2 genes in order to find better chromosom. Crossover increases accuracy faster but it might not reach answer at the end or even reach it after a long time. My solution for getting out of local maximums is increasing mutation probability and it worked for me.

#### Elitism:
The last part is to choose best part of population and put them in the next generation without any change. This will happen using elitismP probabilty between 0.1 to 0.25.

In [2]:
class Chromosome(object):
    def __init__(self, string, encoded, dictionary):
        self.data = string
        self.fitness = decodeText(string, encoded, dictionary)
class Decoder(object):
    def __init__(self, encoded_txt):
        self.encoded = encoded_txt
        self.numOfWords = self.wordsCounter()
        self.dictionary = self.createDictionary("global_text.txt")
        self.populationSize = 600
        self.crossoverP = 0.65
        self.elitismP = 0.2
        self.mutationP = 0.4
        self.population = self.makePopulation()
    def makePopulation(self):
        population = []
        letters = string.ascii_lowercase
        for i in range(self.populationSize):
            randomChromosome = Chromosome(''.join(random.sample(letters, 26)), self.encoded, self.dictionary)
            population.append(randomChromosome)
        return population    
    def wordsCounter(self):
        counter = 0
        words = re.split('\W', self.encoded)
        for word in words:
            if word:
                counter += 1
        return (counter - 1)
    def createDictionary(self, dictionaryAddress):
        text = open(dictionaryAddress).read()
        text = text.lower()
        temp = re.split('\W', text)
        result = set()
        for item in temp:
            result.add(hash(item))
        return result
    def finalDecode(self, chromosome):
        result = ""
        for character in self.encoded:
            if character.isalpha():
                if character.islower():
                    result += chromosome[ord(character) - ord('a')]
                else:
                    result += (chromosome[ord(character) - ord('A')]).upper()
            else:
                result += character
        return result
    def mutate(self, chromosome):
        numbers = []
        while not (len(numbers) == 2):
            temp = random.randint(0, 25)
            if not temp in numbers:
                numbers.append(temp)
        l = list(chromosome.data)
        l[numbers[0]], l[numbers[1]] = l[numbers[1]], l[numbers[0]]
        return Chromosome(''.join(l), self.encoded, self.dictionary)
    def crossover(self):
        crossoverResults = []
        selectedChromosomes = []
        probabalities = []
        for i in range(self.populationSize):
            for j in range(self.population[i].fitness):
                probabalities.append(i)
        for i in range(self.populationSize - int(self.populationSize * self.elitismP)):
            randomIndex = probabalities[random.randint(0, len(probabalities) - 1)]
            selectedChromosomes.append(self.population[randomIndex])
        for i in range(0, len(selectedChromosomes), 2):
            if random.random() < self.crossoverP:
                crossoverResults.append(selectedChromosomes[i])
                crossoverResults.append(selectedChromosomes[i + 1])
            else:
                p1 = list(selectedChromosomes[i].data)
                p2 = list(selectedChromosomes[i + 1].data)
                numbers = []
                while not (len(numbers) == 2):
                    temp = random.randint(0, 25)
                    if not temp in numbers:
                        numbers.append(temp)
                numbers.sort()
                c1 = list(selectedChromosomes[i].data)
                c2 = list(selectedChromosomes[i + 1].data)
                k1 = p1[numbers[0]:numbers[1]]
                k2 = p2[numbers[0]:numbers[1]]
                for k in range(numbers[1] - numbers[0]):
                    index1 = c1.index(k1[k])
                    index2 = c2.index(k2[k])
                    c1[index1] = c1[numbers[0] + k]
                    c1[numbers[0] + k] = k1[k]
                    c2[index2] = c2[numbers[0] + k]
                    c2[numbers[0] + k] = k2[k]
                crossoverResults.append(Chromosome(''.join(c1), self.encoded, self.dictionary))
                crossoverResults.append(Chromosome(''.join(c2), self.encoded, self.dictionary))
        return crossoverResults
    
    def decode(self):
        c = 0
        while (True):
            c += 1
            if (c % 50) == 1:
                maximum = 0
                for item in self.population:
                    if item.fitness > maximum:
                        maximum = item.fitness
                print("Maximum Fitness in generation", c, "is :", maximum)
            for j in range(self.populationSize):
                if (self.population[j].fitness == self.numOfWords):
                    for i in range(26):
                        print(chr(ord('a') + i), ':', self.population[j].data[i])
                    return self.finalDecode(self.population[j].data)
            self.population.sort(key = operator.attrgetter('fitness'))
            newGeneration = []
            for i in range(1, int(self.populationSize * self.elitismP) + 1):
                newGeneration.append(self.population[-i])
            crossoverResults = self.crossover()
            for i in range(len(crossoverResults)):
                if random.random() < self.mutationP:
                    newGeneration.append(self.mutate(crossoverResults[i]))
                else:
                    newGeneration.append(crossoverResults[i])
            self.population = newGeneration

In [3]:
encoded_text = open("encoded_text.txt").read()
d = Decoder(encoded_text)
start = time.clock()
decoded_text = d.decode()
finish = time.clock()
print("It took", finish - start, "seconds.")

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


Maximum Fitness in generation 1 is : 110
Maximum Fitness in generation 51 is : 221
Maximum Fitness in generation 101 is : 385
Maximum Fitness in generation 151 is : 701
a : o
b : r
c : s
d : f
e : w
f : m
g : b
h : t
i : i
j : z
k : g
l : h
m : k
n : n
o : v
p : e
q : l
r : p
s : d
t : j
u : c
v : u
w : y
x : q
y : a
z : x
It took 197.18353630000001 seconds.


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """


In [4]:
print("Decode text is :")
print(decoded_text)

Decode text is :
This response originally fell into a bit bucket.  I'm reposting it
just so Bill doesn't think I'm ignoring him.

In article <C4w5pv.JxD@darkside.osrhe.uoknor.edu> bil@okcforum.osrhe.edu (Bill Conner) writes:
>Jim Perry (perry@dsinc.com) wrote:
>
>[Some stuff about Biblical morality, though Bill's quote of me had little
> to do with what he goes on to say]

Bill,

I'm sorry to have been busy lately and only just be getting around to
this.

Apparently you have some fundamental confusions about atheism; I think
many of these are well addressed in the famous FAQ.  Your generalisms
are then misplaced -- atheism needn't imply materialism, or the lack
of an absolute moral system.  However, I do tend to materialism and
don't believe in absolute morality, so I'll answer your questions.

>How then can an atheist judge value? 

An atheist judges value in the same way that a theist does: according
to a personal understanding of morality.  That I don't believe in an
absolute one do

### Conclusion
This program reaches the answer under 5 minutes (checked it 5 times) and it always reaches the correct answer because it is not possible to reach another solution that makes sensibile words for us  and also because we know that every word in original text is available in our dictionary. The bottleneck of this algorithm is fitness function and compilation time of it will deacrease greatfully if you choose a wise fitness function.